看看均线策略。

In [ ]:
import pandas as pd
from datetime import datetime
import trdb2py

isStaticImg = False
width = 960
height = 768

pd.options.display.max_columns = None
pd.options.display.max_rows = None

trdb2cfg = trdb2py.loadConfig('./trdb2.yaml')

In [ ]:
# 具体基金
asset = 'bitmex.XBTUSD|5m'

# 起始时间，0表示从最开始算起
tsStart = 0
tsStart = int(trdb2py.str2timestamp('2021-03-01', '%Y-%m-%d'))

# 结束时间，-1表示到现在为止
tsEnd = -1
# tsEnd = int(trdb2py.str2timestamp('2019-01-01', '%Y-%m-%d'))

# 初始资金池
paramsinit = trdb2py.trading2_pb2.InitParams(
    money=10000,
)

# 买入参数，用全部的钱来买入（也就是复利）
paramsbuy = trdb2py.trading2_pb2.BuyParams(
    perHandMoney=1,
    nextTimes=1,
    perLimitPrice=1.001,
)

# 卖出参数，全部卖出
paramssell = trdb2py.trading2_pb2.SellParams(
    perVolume=1,
    nextTimes=1,
    perLimitPrice=0.999,    
)

In [ ]:
# baseline    
s0 = trdb2py.trading2_pb2.Strategy(
    name="normal",
    asset=trdb2py.str2asset(asset),         
)
        
buy0 = trdb2py.trading2_pb2.CtrlCondition(
    name='buyandhold',
)

paramsbuy1 = trdb2py.trading2_pb2.BuyParams(
    perHandMoney=1,
)

paramsinit1 = trdb2py.trading2_pb2.InitParams(
    money=10000,
)

s0.buy.extend([buy0])
s0.paramsBuy.CopyFrom(paramsbuy1)
s0.paramsInit.CopyFrom(paramsinit1)        
p0 = trdb2py.trading2_pb2.SimTradingParams(
    assets=[trdb2py.str2asset(asset)],
    startTs=tsStart,
    endTs=tsEnd,
    strategies=[s0],
    title='baseline',
)  

pnlBaseline = trdb2py.simTrading(trdb2cfg, p0)
# trdb2py.showPNL(pnlBaseline, dtFormat='%Y-%m-%d %H:%M:%S', toImg=isStaticImg, width=width, height=height)
trdb2py.showPNL(pnlBaseline, dtFormat='%Y-%m-%d %H:%M:%S', toImg=isStaticImg, width=width, height=height)

In [ ]:
lstparams = []

for ema in range(601, 721):    
    buy0 = trdb2py.trading2_pb2.CtrlCondition(
        name='indicatorsp',
        operators=['upcross'],
        strVals=['ema.{}'.format(ema)],
    )
    
    buy1 = trdb2py.trading2_pb2.CtrlCondition(
        name='waittostart',
        vals=[ema],
    )    
            
    sell0 = trdb2py.trading2_pb2.CtrlCondition(
        name='indicatorsp',
        operators=['downcross'],
        strVals=['ema.{}'.format(ema)],
    )
    
    s0 = trdb2py.trading2_pb2.Strategy(
        name="normal",
        asset=trdb2py.str2asset(asset),
    )

    s0.buy.extend([buy0, buy1])
    s0.sell.extend([sell0])
    s0.paramsBuy.CopyFrom(paramsbuy)
    s0.paramsSell.CopyFrom(paramssell) 
    s0.paramsInit.CopyFrom(paramsinit)        
    lstparams.append(trdb2py.trading2_pb2.SimTradingParams(
        assets=[trdb2py.str2asset(asset)],
        startTs=tsStart,
        endTs=tsEnd,
        strategies=[s0],
        title='ema{}'.format(ema),
    ))
 
lstpnlmix = trdb2py.simTradings(trdb2cfg, lstparams, ignoreTotalReturn=20)

trdb2py.showPNLs(lstpnlmix + [pnlBaseline], dtFormat='%Y-%m-%d %H:%M:%S', toImg=isStaticImg, width=width, height=height)

In [ ]:
dfpnl1 = trdb2py.buildPNLReport(lstpnlmix + [pnlBaseline])

# dfpnl1 = dfpnl[dfpnl['totalReturns'] >= 20]

dfpnl1[['title', 'maxDrawdown', 'maxDrawdownStart', 'maxDrawdownEnd', 'totalReturns', 'sharpe', 'annualizedReturns', 'annualizedVolatility', 'variance']].sort_values(by='totalReturns', ascending=False)

In [ ]:
# 起始时间，0表示从最开始算起
tsStart = 0
tsStart = int(trdb2py.str2timestamp('2021-02-01', '%Y-%m-%d'))

# 结束时间，-1表示到现在为止
tsEnd = -1
tsEnd = int(trdb2py.str2timestamp('2021-03-01', '%Y-%m-%d'))

# baseline    
s0 = trdb2py.trading2_pb2.Strategy(
    name="normal",
    asset=trdb2py.str2asset(asset),         
)
        
buy0 = trdb2py.trading2_pb2.CtrlCondition(
    name='buyandhold',
)

# paramsbuy = trdb2py.trading2_pb2.BuyParams(
#     perHandMoney=1,
# )

paramsinit = trdb2py.trading2_pb2.InitParams(
    money=10000,
)

s0.buy.extend([buy0])
s0.paramsBuy.CopyFrom(paramsbuy1)
s0.paramsInit.CopyFrom(paramsinit)        
p0 = trdb2py.trading2_pb2.SimTradingParams(
    assets=[trdb2py.str2asset(asset)],
    startTs=tsStart,
    endTs=tsEnd,
    strategies=[s0],
    title='baseline',
)  

pnlBaseline1 = trdb2py.simTrading(trdb2cfg, p0)
# trdb2py.showPNL(pnlBaseline, dtFormat='%Y-%m-%d %H:%M:%S', toImg=isStaticImg, width=width, height=height)
trdb2py.showPNL(pnlBaseline1, dtFormat='%Y-%m-%d %H:%M:%S', toImg=isStaticImg, width=width, height=height)

In [ ]:
# 买入参数，用全部的钱来买入（也就是复利）
paramsbuy = trdb2py.trading2_pb2.BuyParams(
    perHandMoney=1,
    nextTimes=1,
    perLimitPrice=1.001,
)

# 卖出参数，全部卖出
paramssell = trdb2py.trading2_pb2.SellParams(
    perVolume=1,
    nextTimes=1,
    perLimitPrice=0.999,    
)

lstparams = []

for ema in range(75, 76):    
    buy0 = trdb2py.trading2_pb2.CtrlCondition(
        name='indicatorsp',
        operators=['upcross'],
        strVals=['ema.{}'.format(ema)],
    )
    
    buy1 = trdb2py.trading2_pb2.CtrlCondition(
        name='waittostart',
        vals=[ema],
    )    
            
    sell0 = trdb2py.trading2_pb2.CtrlCondition(
        name='indicatorsp',
        operators=['downcross'],
        strVals=['ema.{}'.format(ema)],
    )
    
    s0 = trdb2py.trading2_pb2.Strategy(
        name="normal",
        asset=trdb2py.str2asset(asset),
    )

    s0.buy.extend([buy0, buy1])
    s0.sell.extend([sell0])
    s0.paramsBuy.CopyFrom(paramsbuy)
    s0.paramsSell.CopyFrom(paramssell) 
    s0.paramsInit.CopyFrom(paramsinit)        
    lstparams.append(trdb2py.trading2_pb2.SimTradingParams(
        assets=[trdb2py.str2asset(asset)],
        startTs=tsStart,
        endTs=tsEnd,
        strategies=[s0],
        title='ema{}'.format(ema),
    ))
 
lstpnlmix1 = trdb2py.simTradings(trdb2cfg, lstparams, ignoreTotalReturn=20)

trdb2py.showPNLs(lstpnlmix1 + [pnlBaseline1], dtFormat='%Y-%m-%d %H:%M:%S', toImg=isStaticImg, isShowBuy=True, isShowSell=True, width=width, height=height)

In [ ]:
dfctrl = trdb2py.buildPNLCtrlData(lstpnlmix1[0]['pnl'], dtFormat='%Y-%m-%d %H:%M:%S')
dfctrl

In [ ]:
lstparams = []

for ema0 in range(5, 61):
    for ema1 in range(ema0, 61):
        buy0 = trdb2py.trading2_pb2.CtrlCondition(
            name='indicatorsp',
            operators=['upcross'],
            strVals=['ema.{}'.format(ema0)],
        )

        buy1 = trdb2py.trading2_pb2.CtrlCondition(
            name='waittostart',
            vals=[ema0],
        )
        
        buy2 = trdb2py.trading2_pb2.CtrlCondition(
            name='indicatorsp',
            operators=['up'],
            strVals=['ema.{}'.format(ema1)],
        )
        
        buy3 = trdb2py.trading2_pb2.CtrlCondition(
            name='waittostart',
            vals=[ema1],
        )        

        sell0 = trdb2py.trading2_pb2.CtrlCondition(
            name='indicatorsp',
            operators=['downcross'],
            strVals=['ema.{}'.format(ema0)],
        )

        s0 = trdb2py.trading2_pb2.Strategy(
            name="normal",
            asset=trdb2py.str2asset(asset),
        )

        s0.buy.extend([buy0, buy1, buy2, buy3])
        s0.sell.extend([sell0])
        s0.paramsBuy.CopyFrom(paramsbuy)
        s0.paramsSell.CopyFrom(paramssell) 
        s0.paramsInit.CopyFrom(paramsinit)        
        lstparams.append(trdb2py.trading2_pb2.SimTradingParams(
            assets=[trdb2py.str2asset(asset)],
            startTs=tsStart,
            endTs=tsEnd,
            strategies=[s0],
            title='ema{}&ema{}'.format(ema0, ema1),
        ))
 
# lstpnlmix = trdb2py.simTradings(trdb2cfg, lstparams, ignoreTotalReturn=180)

# trdb2py.showPNLs(lstpnlmix + [pnlBaseline], dtFormat='%Y-%m-%d %H:%M:%S', toImg=isStaticImg, width=width, height=height)

In [ ]:
dfpnl = trdb2py.buildPNLReport(lstpnlmix + [pnlBaseline])

dfpnl1 = dfpnl[dfpnl['totalReturns'] >= 1]

dfpnl1[['title', 'maxDrawdown', 'maxDrawdownStart', 'maxDrawdownEnd', 'totalReturns', 'sharpe', 'annualizedReturns', 'annualizedVolatility', 'variance']].sort_values(by='totalReturns', ascending=False)